In [1]:
import pandas as pd
from pandas import DataFrame
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision.datasets import MNIST
# from torchvision import transforms
import pytorch_lightning as pl
import math
import numpy as np

In [2]:
# # Import Kaggle data
# import kaggle
# kagggle competitions download tabular-playground-series-mar-2021

In [3]:
# Add data to memory for discovery
train,test = pd.read_csv('./train.csv'),pd.read_csv('./test.csv')

In [4]:
# Get data structure
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cat0    300000 non-null  object 
 2   cat1    300000 non-null  object 
 3   cat2    300000 non-null  object 
 4   cat3    300000 non-null  object 
 5   cat4    300000 non-null  object 
 6   cat5    300000 non-null  object 
 7   cat6    300000 non-null  object 
 8   cat7    300000 non-null  object 
 9   cat8    300000 non-null  object 
 10  cat9    300000 non-null  object 
 11  cat10   300000 non-null  object 
 12  cat11   300000 non-null  object 
 13  cat12   300000 non-null  object 
 14  cat13   300000 non-null  object 
 15  cat14   300000 non-null  object 
 16  cat15   300000 non-null  object 
 17  cat16   300000 non-null  object 
 18  cat17   300000 non-null  object 
 19  cat18   300000 non-null  object 
 20  cont0   300000 non-null  float64
 21  cont1   30

In [5]:
# Peek at data examples
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [6]:
# Get number of unique categories
cal_cols = train.select_dtypes('object')
cal_cols_keys = cal_cols.keys()
# cal_cols_vals = np.array((cal_cols.values))

#Build sample item
cat_stats = np.array([train[c].nunique() for c in cal_cols])
median_cat = math.ceil(np.median(cat_stats))
target_cat_index = int(np.where(cat_stats == median_cat)[0])
target_cat_name = cal_cols_keys[target_cat_index]

print("Categroy with median total unique values:\n",target_cat_name,": ",train[target_cat_name].nunique(),"unique values")

Categroy with median total unique values:
 cat3 :  13 unique values


In [7]:
# Index categories
categoricals_to_ix = {}

for c in train.select_dtypes('object'):
    categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}

#Confirm with sample indexing
sample_cat = categoricals_to_ix[target_cat_name]
print(target_cat_name,"indices:\n",sample_cat)



cat3 indices:
 {'B': 0, 'A': 1, 'C': 2, 'D': 3, 'G': 4, 'N': 5, 'H': 6, 'F': 7, 'E': 8, 'K': 9, 'I': 10, 'J': 11, 'L': 12}


In [8]:
# Test embeddings
embeds = {}

for cix_key in categoricals_to_ix.keys():
    cix = categoricals_to_ix[cix_key]
    bag_size = len(cix)
    embed_size = math.isqrt(bag_size) #hyperparameter to optimize later
    embeds[cix_key] = nn.Embedding(bag_size,embed_size)

#Confirm with sample embedding
target_cat_keys = list(sample_cat.keys())
target_cat_vals = np.array(list(sample_cat.values()))
target_cat_vals_median = math.ceil(np.median(target_cat_vals))
target_cat_keys_sample = target_cat_keys[target_cat_vals_median]
print("Sanple",target_cat_name,"category:\n",target_cat_keys_sample)
    
testor = torch.tensor([categoricals_to_ix[target_cat_name][target_cat_keys_sample]], dtype=torch.long)
cat_keys_sample_hash = embeds[target_cat_name](testor)
print("Sanple category embedding:\n",cat_keys_sample_hash)

Sanple cat3 category:
 H
Sanple category embedding:
 tensor([[-0.9979, -1.6842, -1.8397]], grad_fn=<EmbeddingBackward>)


In [15]:
# Define embeddings content (dictionary and translations)
#function to build embeddings library
#function assumes only categorical features are fed in
def BuildEmbedlibrary(features):
    categoricals_to_ix,embeds = {},{}
    
    #Build category index
    for c in features:
        categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}
    
    #Create embeddings for each feature
    for cix_key in categoricals_to_ix.keys():
        cix = categoricals_to_ix[cix_key]
        bag_size = len(cix)
        embed_size = math.isqrt(bag_size) #hyperparameter to optimize later
        embeds[cix_key] = nn.Embedding(bag_size,embed_size)
    
    return categoricals_to_ix,embeds

#function that gets respective embedding for given feature value
def CatToSubvector(category,feature_name,categorical_library,embedding_library):
    [print(valu,"\n") for valu in [category,feature_name,categorical_library,embedding_library]]
    raw_idx = torch.tensor([categorical_library[feature_name][category]], dtype=torch.long)
    subvector = embedding_library[feature_name](raw_idx)
    
    return subvector

In [16]:
# Confirm function structure
test_cat_dict,test_embeddings = BuildEmbedlibrary(train.select_dtypes('object'))
test_subvec = CatToSubvector(target_cat_keys_sample,target_cat_name,test_cat_dict,test_embeddings)

print("Sanple category embedding:\n",test_subvec)

H 

cat3 

{'cat0': {'A': 0, 'B': 1}, 'cat1': {'I': 0, 'K': 1, 'A': 2, 'F': 3, 'L': 4, 'N': 5, 'J': 6, 'M': 7, 'O': 8, 'B': 9, 'H': 10, 'G': 11, 'C': 12, 'D': 13, 'E': 14}, 'cat2': {'A': 0, 'G': 1, 'C': 2, 'O': 3, 'D': 4, 'F': 5, 'Q': 6, 'J': 7, 'L': 8, 'I': 9, 'M': 10, 'H': 11, 'U': 12, 'N': 13, 'R': 14, 'E': 15, 'K': 16, 'S': 17, 'B': 18}, 'cat3': {'B': 0, 'A': 1, 'C': 2, 'D': 3, 'G': 4, 'N': 5, 'H': 6, 'F': 7, 'E': 8, 'K': 9, 'I': 10, 'J': 11, 'L': 12}, 'cat4': {'B': 0, 'E': 1, 'H': 2, 'I': 3, 'D': 4, 'F': 5, 'G': 6, 'M': 7, 'K': 8, 'J': 9, 'T': 10, 'C': 11, 'L': 12, 'P': 13, 'S': 14, 'O': 15, 'N': 16, 'Q': 17, 'A': 18, 'R': 19}, 'cat5': {'BI': 0, 'AB': 1, 'BU': 2, 'M': 3, 'T': 4, 'K': 5, 'L': 6, 'CG': 7, 'BG': 8, 'CI': 9, 'N': 10, 'G': 11, 'X': 12, 'Q': 13, 'O': 14, 'BO': 15, 'BB': 16, 'BX': 17, 'AF': 18, 'BA': 19, 'BQ': 20, 'CA': 21, 'D': 22, 'AQ': 23, 'AS': 24, 'AW': 25, 'BE': 26, 'CK': 27, 'AL': 28, 'BK': 29, 'AT': 30, 'CL': 31, 'C': 32, 'CF': 33, 'I': 34, 'AH': 35, 'CD': 36, 'A

In [17]:
# Define data

#define dataset
class ClaimsDataset(Dataset):
    def __init__(self, csv_path='./train.csv'):
        """
        Args:
            raw_data (dataframe): Dataframe with raw featurres and labels.
        """
#         self.categoricals,self.embeddings = {},{}
        raw_data = pd.read_csv(csv_path)
        
#         self.l = len(raw_data)
#         self.target = raw_data.pop('target')        
        categorical_features = raw_data.select_dtypes(include='object')
        noncategorical_features = raw_data.select_dtypes(exclude='object')
        
        self.categoricals,self.embeddings = BuildEmbedlibrary(categorical_features)
        
        subvects = []
        for row in categorical_features:
            subsubvects = []
            for v,c in zip(row,categorical_features.columns):
                subsubvects.append(CatToSubvector(v,c,self.categoricals,self.embeddings))
            subvects.append(torch.cat(subsubvects))
        
        self.vectors = torch.cat([
            subvects,
            torch.tensor(noncategorical_features.values)
        ])
        
        print(self.vectors)
    def __len__(self):
        return len(self.vectors)

In [18]:
cd = ClaimsDataset()
print(cd.categoricals,"\n",cd.embeddings,"\n",cd.target)

c 

cat0 

{'cat0': {'A': 0, 'B': 1}, 'cat1': {'I': 0, 'K': 1, 'A': 2, 'F': 3, 'L': 4, 'N': 5, 'J': 6, 'M': 7, 'O': 8, 'B': 9, 'H': 10, 'G': 11, 'C': 12, 'D': 13, 'E': 14}, 'cat2': {'A': 0, 'G': 1, 'C': 2, 'O': 3, 'D': 4, 'F': 5, 'Q': 6, 'J': 7, 'L': 8, 'I': 9, 'M': 10, 'H': 11, 'U': 12, 'N': 13, 'R': 14, 'E': 15, 'K': 16, 'S': 17, 'B': 18}, 'cat3': {'B': 0, 'A': 1, 'C': 2, 'D': 3, 'G': 4, 'N': 5, 'H': 6, 'F': 7, 'E': 8, 'K': 9, 'I': 10, 'J': 11, 'L': 12}, 'cat4': {'B': 0, 'E': 1, 'H': 2, 'I': 3, 'D': 4, 'F': 5, 'G': 6, 'M': 7, 'K': 8, 'J': 9, 'T': 10, 'C': 11, 'L': 12, 'P': 13, 'S': 14, 'O': 15, 'N': 16, 'Q': 17, 'A': 18, 'R': 19}, 'cat5': {'BI': 0, 'AB': 1, 'BU': 2, 'M': 3, 'T': 4, 'K': 5, 'L': 6, 'CG': 7, 'BG': 8, 'CI': 9, 'N': 10, 'G': 11, 'X': 12, 'Q': 13, 'O': 14, 'BO': 15, 'BB': 16, 'BX': 17, 'AF': 18, 'BA': 19, 'BQ': 20, 'CA': 21, 'D': 22, 'AQ': 23, 'AS': 24, 'AW': 25, 'BE': 26, 'CK': 27, 'AL': 28, 'BK': 29, 'AT': 30, 'CL': 31, 'C': 32, 'CF': 33, 'I': 34, 'AH': 35, 'CD': 36, 'A

KeyError: 'c'